# Setup

In [2]:
from FableAPI.fable_init import api
api.setup(blocking=True)
moduleids = api.discoverModules()
print("Module IDs: ", moduleids)
moduleID = moduleids[0]

Dongle already connected
setup done
Runtime Created BE7 
Module IDs:  ['BE7 ']


# Utils

In [3]:
import camera_tools as ct
import cv2
import numpy as np

def callback(value):
    pass

def setTresh(v1_min, v2_min, v3_min, v1_max, v2_max, v3_max):
    range_filter = "LAB"
    camera = cv2.VideoCapture(0)
    cv2.namedWindow("Trackbars", 0)
    for i in ["MIN", "MAX"]:
        v = 0 if i == "MIN" else 255
        for j in range_filter:
            cv2.createTrackbar("%s_%s" % (j, i), "Trackbars", v, 255, callback)
    ret, image = camera.read()
    while not ret:
        ret, image = camera.read()
    frame_to_thresh = cv2.cvtColor(image, cv2.COLOR_BGR2LAB) # LAB
    thresh = cv2.inRange(frame_to_thresh, (v1_min, v2_min, v3_min), (v1_max, v2_max, v3_max))
    image = cv2.flip(image, 1)
    cv2.imshow("Original", image)
    thresh = cv2.flip(thresh, 1)
    cv2.imshow("Thresh", thresh)
    camera.release()
    cv2.destroyAllWindows()
    return np.array([v1_min, v2_min, v3_min]), np.array([v1_max, v2_max, v3_max])

# To test the camera and object detection algorithm:
def detectObj():
    cam = ct.prepare_camera()
    while True:
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        if x is not None:
            break
    print("Now the camera is done adjusting!")

    for _ in range(10):
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        print(x, y)
        
    cam.release()

# Grab an image and locate the largest green object:
def getPos():
    cam = ct.prepare_camera()
    while True:
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        if x is not None:
            break
    # print("Now the camera is done adjusting!")
    X,Y = [],[] 
    for _ in range(10):
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        # print(x, y)
        X.append(x)
        Y.append(y)
    cam.release()
    X = np.rint(np.mean(np.asarray(X))).astype(int)
    Y = np.rint(np.mean(np.asarray(Y))).astype(int)
    return (X,Y)

# Define position over time
def posOverTime():
    cam = ct.prepare_camera()
    class TestClass:
        def __init__(self):
            self.i = 0
        @ct.camera_loop(cam, wait_time=1.0) # only runs the function every 0.5 second.
        def go(self, x, y, clickpoint):
            if self.i > 100:
                return True
            print(self.i, x, y)
            self.i += 1
            return False
    test = TestClass()
    test.go()

def showCamera():
    cam = ct.prepare_camera()
    ct.show_camera(cam)

def move(x,y, module = moduleID, getPixel = True):
    api.setPos(x, y, module)
    api.sleep(2)
    if getPixel:
        return getPos()



def selectPoint():
    cam = ct.prepare_camera()
    
    class TestClass:
        def __init__(self):
            self.i = 0

        @ct.camera_loop(cam,low_green, high_green, wait_time=5.0) # only runs the function every 0.5 second.
        def go(self, x, y, clickpoint):
            if clickpoint is not None:
                print(clickpoint)
                xy = torch.tensor(clickpoint).float()
                xy = model(xy)
                print(xy[0],xy[1])
                print("\n")
                move(xy[0],xy[1],getPixel = False)
            if self.i > 1000:
                return True
            self.i += 1
            return False
            
    test = TestClass()
    test.go()

import torch
# # Define neural network
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))
        x = self.predict(x)
        return x

# CALIBRATE CAMERA

In [5]:
# Set treshold camera
# low_green, high_green = setTresh(25,87,64,233,109,153)
low_green, high_green = ct.colorpicker()
# [0 0 0] [255  88 255]
    

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to tar

Create training data

In [6]:
stepX = 30
stepY = 30
angleY = 0
odd = True
x,y = [],[]

def appendL(x,list):
    for el in list:
        x.append(el)
    return x

while angleY >= -90:
    if odd:
        x1 = range(-90,90+stepX,stepX)
        y1 = [angleY]*len(x1)
        odd = False
    else:
        x1 = range(90,-90-stepX,-stepX)
        y1 = [angleY]*len(x1)
        odd = True
    angleY -= stepY
    x = appendL(x,x1)
    y = appendL(y,y1)

knownPos = []
move(0,0)
showCamera()

[-90, -60, -30, 0, 30, 60, 90, 90, 60, 30, 0, -30, -60, -90, -90, -60, -30, 0, 30, 60, 90, 90, 60, 30, 0, -30, -60, -90]
[0, 0, 0, 0, 0, 0, 0, -30, -30, -30, -30, -30, -30, -30, -60, -60, -60, -60, -60, -60, -60, -90, -90, -90, -90, -90, -90, -90]


In [9]:
## TO SAVE STUFF

# import pickle
# with open('xy_pixel_Y30.pickle', 'wb') as output:
#     pickle.dump(knownPos, output)
# with open('motor_angles_Y30.pickle', 'wb') as output:
#     pickle.dump(list(zip(x,y)), output)

In [10]:
# import torch
# X = torch.tensor(knownPos).float()
# t = torch.tensor(list(zip(x,y))).float()

# # # Define neural network
# class Net(torch.nn.Module):
#     def __init__(self, n_feature, n_hidden, n_output):
#         super(Net, self).__init__()
#         self.hidden = torch.nn.Linear(n_feature, n_hidden)
#         self.predict = torch.nn.Linear(n_hidden, n_output)

#     def forward(self, x):
#         x = torch.sigmoid(self.hidden(x))
#         x = self.predict(x)
#         return x

# model = Net(n_feature=2, n_hidden=20, n_output=2)
# loss_func = torch.nn.MSELoss()
# # optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# # loop for train
# num_epochs = 200000
# for _ in range(num_epochs):
#     prediction = model(X) # Forward pass prediction. Saves intermediary values required for backwards pass
#     loss = loss_func(prediction, t) # Computes the loss for each example, using the loss function defined above
#     optimizer.zero_grad() # Clears gradients from previous iteration
#     loss.backward() # Backpropagation of errors through the network
#     optimizer.step() # Updating weights

#     #print("Prediction: ", prediction.detach().numpy())
#     print("Loss: ", loss.detach().numpy())
# torch.save(model.state_dict(), 'stepY30_model_20_001.pth')

In [11]:
# model = Net(n_feature=2, n_hidden=2, n_output=2)
# loss_func = torch.nn.MSELoss()
# # optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# # loop for train
# num_epochs = 200000
# for _ in range(num_epochs):
#     prediction = model(X) # Forward pass prediction. Saves intermediary values required for backwards pass
#     loss = loss_func(prediction, t) # Computes the loss for each example, using the loss function defined above
#     optimizer.zero_grad() # Clears gradients from previous iteration
#     loss.backward() # Backpropagation of errors through the network
#     optimizer.step() # Updating weights

#     #print("Prediction: ", prediction.detach().numpy())
#     print("Loss: ", loss.detach().numpy())
# torch.save(model.state_dict(), 'stepY30_model_2_01.pth')    

In [12]:
X = torch.tensor(knownPos).float()
t = torch.tensor(list(zip(x,y))).float()

model = Net(n_feature=2, n_hidden=50, n_output=2)
loss_func = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# loop for train
num_epochs = 100000
for _ in range(num_epochs):
    prediction = model(X) # Forward pass prediction. Saves intermediary values required for backwards pass
    loss = loss_func(prediction, t) # Computes the loss for each example, using the loss function defined above
    optimizer.zero_grad() # Clears gradients from previous iteration
    loss.backward() # Backpropagation of errors through the network
    optimizer.step() # Updating weights

    #print("Prediction: ", prediction.detach().numpy())
    print("Loss: ", loss.detach().numpy())

torch.save(model.state_dict(), 'last_stepY30_model_50_001.pth')

Loss:  3339.8496
Loss:  3338.1003
Loss:  3336.3904
Loss:  3334.7239
Loss:  3333.0718
Loss:  3331.418
Loss:  3329.7734
Loss:  3328.154
Loss:  3326.5608
Loss:  3324.9905
Loss:  3323.437
Loss:  3321.891
Loss:  3320.3403
Loss:  3318.7817
Loss:  3317.2188
Loss:  3315.6506
Loss:  3314.0754
Loss:  3312.483
Loss:  3310.865
Loss:  3309.2192
Loss:  3307.559
Loss:  3305.9011
Loss:  3304.252
Loss:  3302.598
Loss:  3300.908
Loss:  3299.1655
Loss:  3297.3967
Loss:  3295.658
Loss:  3293.9846
Loss:  3292.3687
Loss:  3290.782
Loss:  3289.2031
Loss:  3287.619
Loss:  3286.0212
Loss:  3284.4
Loss:  3282.7415
Loss:  3281.0396
Loss:  3279.3152
Loss:  3277.6265
Loss:  3276.0142
Loss:  3274.4666
Loss:  3272.9534
Loss:  3271.4575
Loss:  3269.97
Loss:  3268.4866
Loss:  3267.0046
Loss:  3265.5198
Loss:  3264.0251
Loss:  3262.4983
Loss:  3260.9224
Loss:  3259.3044
Loss:  3257.6714
Loss:  3256.0474
Loss:  3254.4407
Loss:  3252.842
Loss:  3251.241
Loss:  3249.6438
Loss:  3248.0508
Loss:  3246.4534
Loss:  3244.8423


In [13]:
selectPoint()

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to target thread (0x55e66ed355b0)

QObject::moveToThread: Current thread (0x55e66ed355b0) is not the object's thread (0x55e66ef6c940).
Cannot move to tar

[496, 287]
tensor(60.0006, grad_fn=<SelectBackward0>) tensor(-44.9999, grad_fn=<SelectBackward0>)


[496, 287]
tensor(60.0006, grad_fn=<SelectBackward0>) tensor(-44.9999, grad_fn=<SelectBackward0>)


[496, 287]
tensor(60.0006, grad_fn=<SelectBackward0>) tensor(-44.9999, grad_fn=<SelectBackward0>)


[496, 287]
tensor(60.0006, grad_fn=<SelectBackward0>) tensor(-44.9999, grad_fn=<SelectBackward0>)


[496, 287]
tensor(60.0006, grad_fn=<SelectBackward0>) tensor(-44.9999, grad_fn=<SelectBackward0>)


[191, 253]
tensor(-63.8469, grad_fn=<SelectBackward0>) tensor(-91.8730, grad_fn=<SelectBackward0>)


[191, 253]
tensor(-63.8469, grad_fn=<SelectBackward0>) tensor(-91.8730, grad_fn=<SelectBackward0>)


[191, 253]
tensor(-63.8469, grad_fn=<SelectBackward0>) tensor(-91.8730, grad_fn=<SelectBackward0>)


[191, 253]
tensor(-63.8469, grad_fn=<SelectBackward0>) tensor(-91.8730, grad_fn=<SelectBackward0>)


[191, 253]
tensor(-63.8469, grad_fn=<SelectBackward0>) tensor(-91.8730, grad_fn=<SelectBackward0